<a href="https://colab.research.google.com/github/ProfEngel/FineTuneLLM/blob/main/01_Parseandprepare_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisierung des RunPod-Pods samt Image

Api-Key von Runpod bereitstellen

In [ ]:
#@title 1. RunPod Initialisierung
!wget --quiet --show-progress https://github.com/Run-Pod/runpodctl/releases/download/v1.10.0/runpodctl-linux-amd -O runpodctl
!chmod +x runpodctl
!cp runpodctl /usr/bin/runpodctl

rp_key = input("Gib den Runpod-ApiKey ein")
!runpodctl config --apiKey {rp_key}
#!runpodctl create pod --gpuType "NVIDIA RTX A6000" --secureCloud --gpuCount 2 --templateId bofievo43g --imageName ghcr.io/huggingface/text-generation-inference:latest

!runpodctl create pod \
--gpuType "NVIDIA RTX A6000" \
--secureCloud \
--gpuCount 2 \
--templateId bofievo43g \
--imageName ghcr.io/huggingface/text-generation-inference:latest \
--containerDiskSize 100 \
--volumeSize 100 \
--volumePath "/workspace" \
--ports "8080/http" \
--args "--model-id TheBloke/Llama-2-70B-chat-AWQ --trust-remote-code --port 8080 --max-input-length 2048 --max-total-tokens 4096 --max-batch-prefill-tokens 4096 --quantize awq --speculate 3"

runpodctl           100%[===================>]  11.81M  54.5MB/s    in 0.2s    
Gib den Runpod-ApiKey einAY7OXSUOGY41VK8IQIRYGYHI3Z59E6NETY8852B0
saved apiKey into config file: /root/.runpod.yaml
pod "gpcq1dt23kg6qg" created for $1.580 / hr


In [ ]:
#@title 2. Initialisierung der Bibs
%%capture
# Überprüfung, ob eine requirements.txt-Datei bereits existiert und deren Löschung
!if [ -f requirements.txt ]; then rm requirements.txt; fi
# Herunterladen der neuen requirements.txt-Datei
!wget https://raw.githubusercontent.com/ProfEngel/FineTuneLLM/main/requirements.txt
# Installation der Abhängigkeiten aus der heruntergeladenen requirements.txt-Datei
!pip install -r requirements.txt

In [ ]:
#@title 3. Abfrage der relevanten Daten
from google.colab import files
import os

# Erstellen Sie den Ordner 'data', falls er nicht existiert
if not os.path.exists('data'):
    os.makedirs('data')

# Fordern Sie den Benutzer zur Hochladung einer Datei auf
uploaded = files.upload()

# Speichern Sie die hochgeladene Datei unter 'data/train.pdf'
for filename in uploaded.keys():
    with open(f'data/train.pdf', 'wb') as f:
        f.write(uploaded[filename])
    print(f"'{filename}' wurde als 'data/train.pdf' gespeichert.")
    break  # Beendet die Schleife nach der Speicherung der ersten hochgeladenen Datei

# Benutzereingaben anfordern
kontext_laenge = input("Wie lange soll die Kontext-Länge sein? (Empfohlen: 1000) ")
kontext = input("Worum geht es in diesem Datensatz? (Bsp. 'Entscheidungsökonomik. Definitionen, Ablauf und Verständnisfragen. Auch mit Beispielen') ")

# Konvertieren Sie die Eingabe für die Kontextlänge in einen Integer, falls nötig
try:
    kontext_laenge = int(kontext_laenge)
except ValueError:
    print("Die eingegebene Kontextlänge ist keine gültige Zahl. Es wird der empfohlene Wert 1000 verwendet.")
    kontext_laenge = 1000

# Variablen im Quellcode zuweisen
model_context_length = kontext_laenge
context = f'Context: {kontext}'

# Beispiel für den Benutzer anzeigen
print("Bitte geben Sie ein Trainings- und ein Testbeispiel im folgenden Format ein:")
print("Trainingsbeispiel:")
print("Wie erkennt man ein Nash-Gleichgewicht?\nEin Nash-Gleichgewicht ist erreicht, wenn die Auszahlungen für die Strategien der Spieler in einer Weise kombiniert sind, dass keiner der Spieler durch einseitiges Abweichen von seiner Strategie eine bessere Auszahlung erzielen kann.\n")
print("Testbeispiel:")
print("Was bedeutet das Nash-Gleichgewicht für die Entscheidungsfindung der Spieler?\nEs bedeutet, dass in Abhängigkeit von der Wahl des Gegenspielers ein Spieler die optimale Entscheidung trifft, indem er zwischen allen seinen möglichen Strategien wählt, ohne seine Entscheidung nachträglich ändern zu wollen, da jede Änderung zu einer schlechteren Auszahlung führen würde.\n")

# Funktion zur Formatierung der Eingabe
def format_sample(prompt_type):
    question = input(f"Bitte geben Sie die Frage für Ihr {prompt_type}-Beispiel ein: ")
    answer = input(f"Bitte geben Sie die Antwort für Ihr {prompt_type}-Beispiel ein: ")
    formatted_sample = f"{question}\n{answer}\n"
    return formatted_sample

# Benutzer zur Eingabe für das Trainingsbeispiel auffordern
train_sample = format_sample("Trainings")
print("\nIhr formatiertes Trainingsbeispiel:")
print(train_sample)

# Benutzer zur Eingabe für das Testbeispiel auffordern
test_sample = format_sample("Test")
print("\nIhr formatiertes Testbeispiel:")
print(test_sample)

In [ ]:
import requests
import json

url = f"https://{pod_id}-8080.proxy.runpod.net/generate"
print(url)  # Geben Sie die URL vor der Anfrage aus, um sie zu überprüfen.


payload = {
    "inputs": "[INST] Geben Sie hier Ihren Test-Prompt ein. [/INST]\n\n",
    "parameters": {
        "max_new_tokens": 50,
        "do_sample": False,
        "stop": ["</s>", "[INST]"]
    }
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(payload), headers=headers)

if response.status_code == 200:
    print("Erfolgreiche Verbindung und Antwort:", response.text)
else:
    print(f"Fehler: HTTP-Status {response.status_code}, Antwort: {response.text}")


https://gpcq1dt23kg6qg-8080.proxy.runpod.net/generate
Erfolgreiche Verbindung und Antwort: {"generated_text":"Sure! Here's a prompt for a text-based conversation:\n\nYou: Hi there! How can I help you today?\n\nUser: Hi! I'm looking for a new restaurant to try in the city. Do"}


In [ ]:
#@title 4. Datenübergabe von IDs
import subprocess

!runpodctl get pod
output = subprocess.run(["runpodctl", "get", "pod"], stdout=subprocess.PIPE).stdout.decode("utf-8")
pod_id = output.split()[6]
print(pod_id)
# Pfad zur .env-Datei
env_file_path = ".env"

# Überprüfen, ob die .env-Datei existiert, und sie löschen
if os.path.exists(env_file_path):
    os.remove(env_file_path)
    print(f"Die Datei {env_file_path} wurde gelöscht.")

# Fordern Sie den Nutzer zur Eingabe auf
runpod_pod_id = pod_id
pod_id=runpod_pod_id

# Erstellen Sie die .env-Datei und schreiben Sie die Variable hinein
with open(".env", "w") as file:
    file.write(f"RUNPOD_POD_ID={runpod_pod_id}")

print(".env-Datei erfolgreich erstellt.")

ID            	NAME                                         	GPU        	IMAGE NAME                                          	STATUS  
gpcq1dt23kg6qg	ghcr.io/huggingface/text-generation-inference	2 RTX A6000	ghcr.io/huggingface/text-generation-inference:latest	RUNNING	
gpcq1dt23kg6qg


NameError: name 'os' is not defined

In [ ]:
#@title 5. PDF zu Q&A
import PyPDF2
import os

def pdf_to_text(pdf_path, txt_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    with open(txt_path, 'w') as f:
        f.write(text)

def main():
    summary = []
    paths_to_check = ['data/train.pdf', 'data/test.pdf']

    for pdf_path in paths_to_check:
        # Extract the base name to create txt file name
        base_name = os.path.basename(pdf_path).split('.')[0]
        txt_path = f'data/raw_{base_name}.txt'

        # Check if PDF exists
        if os.path.exists(pdf_path):
            pdf_to_text(pdf_path, txt_path)
            summary.append(f"Converted {pdf_path} to {txt_path}.")
        else:
            summary.append(f"{pdf_path} was not found.")

    # Print summary
    print("Summary:")
    for item in summary:
        print(f"- {item}")

if __name__ == "__main__":
    main()


import os
import time
import openai
from dotenv import load_dotenv
import tiktoken
import requests
import json
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
pod_id = os.getenv("RUNPOD_POD_ID")
print(f"Verwendete POD_ID: {pod_id}")


# context
#model_context_length = 1000 #don't make this too small or the model will be dealing with a lot of chopped input.
#context = 'Context: Stanford Name, Image and Likeness Disclosure Policies.'
tokens_per_question = 30 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question) # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
# chunk_size = 200 # for testing, set a smaller chunk size.

#train_sample = "According to Stanford NIL policies, may a student-athlete miss class to engage in NIL related activity??\nStudent-athletes need to work with their professors to seek approval for any missed class time to engage in NIL activities. Stanford Athletics is not able to provide approval for student-athletes missing class."
#test_sample = "Can entering a NIL deal affect a Stanford student's eligibility for scholarships?\nNo, not as long as students follow California law and Stanford policy."

questions_per_chunk_train = int(chunk_size / tokens_per_question)
questions_per_chunk_test = max(int(questions_per_chunk_train / 10),1)

print(f'Setting {questions_per_chunk_train} questions per {int(chunk_size)}-token chunk for QA train dataset generation.')

def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = sum(1 for _ in encoding.encode(text))
    return token_count

def read_and_chunk_txt(file_path):
    chunks = []
    chunk = ""
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            text = line.strip()
            if count_tokens(chunk + text) > chunk_size:
                chunks.append(chunk.strip())
                chunk = text
            else:
                chunk += " " + text
    if chunk:
        chunks.append(chunk.strip())
    return chunks

def query_runpod(pod_id, prompt, max_tokens):
    url = f"https://{pod_id}-8080.proxy.runpod.net/generate"
    prompt = f'[INST] {prompt} [/INST]\n\n'
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_tokens,
            "do_sample": False,
            "stop": [
                "</s>",
                "[INST]"
            ]
        }
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    # print(url)
    if response.status_code == 200:
        return json.loads(response.text)["generated_text"]
    else:
        return None

load_dotenv()
api_choice = input("Choose the API to use (openai/runpod): ").strip().lower()

if api_choice == "openai":
    openai.api_key = os.getenv("OPENAI_API_KEY")
    if not openai.api_key:
        print("OpenAI API key is missing. Exiting.")
        exit(1)
elif api_choice == "runpod":
    pod_id = os.getenv("RUNPOD_POD_ID")
    if not pod_id:
        print("RunPod Pod ID is missing. Exiting.")
        exit(1)
else:
    print("Invalid API choice. Exiting.")
    exit(1)

chunks = read_and_chunk_txt("data/raw_train.txt")

total_tokens = sum(count_tokens(chunk) for chunk in chunks)
print(f"Total tokens in all chunks: {total_tokens}")

estimated_input_tokens = total_tokens * 1.1
estimated_output_tokens = total_tokens * 50/tokens_per_question
total_estimated_tokens = estimated_input_tokens + estimated_output_tokens

estimated_cost_gpt4 = (estimated_input_tokens / 1000 * 0.03) + (estimated_output_tokens  / 1000 * 0.06)
estimated_cost_gpt35turbo = (estimated_input_tokens / 1000 * 0.003) + (estimated_output_tokens  / 1000 * 0.004)
print(f"Estimated cost with gpt-4: ${estimated_cost_gpt4:.2f}")
print(f"Estimated cost with gpt-3.5-turbo-16k: ${estimated_cost_gpt35turbo:.2f}")

while True:
    process_option = input("Do you want to process one chunk (type 'one') or all chunks (type 'all')? ").strip().lower()
    if process_option in ['one', 'all']:
        break
    else:
        print("Invalid option. Please enter 'one' or 'all'.")

snippets = [
    f"Geben Sie {questions_per_chunk_train} Frage- und Antwortpaare auf der Grundlage des obigen Textes immer in Deutsch an. Die Fragen müssen mit \"Im Kontext von ...\" beginnen. Die Antworten sollten wortwörtlich aus dem obigen Text übernommen werden. Berücksichtigen Sie bei der Formulierung jeder Frage, dass der Leser die anderen Fragen nicht sieht und auch keinen Zugang zu ihnen hat, um den Kontext zu erkennen. Variieren Sie den Stil und das Format der Fragen. Beantworten Sie jede Frage und jede Antwort im Klartext in einer neuen Zeile. Geben Sie keine Fragenummern an. Hier ist ein Beispiel für zwei Frage-Antwort-Paare:\n\n{train_sample}",
        f"Geben Sie {questions_per_chunk_test} Frage- und Antwortpaar(e) auf der Grundlage des obigen Textes an. Die Fragen müssen mit \"Im Kontext von ...\" beginnen. Die Antworten sollten NICHT wortwörtlich aus dem obigen Text übernommen werden, aber sie sollten die Bedeutung beibehalten. Berücksichtigen Sie bei der Formulierung jeder Frage, dass der Leser die anderen Fragen nicht sieht und auch keinen Zugang zu ihnen hat, um den Kontext zu verstehen. Variieren Sie den Stil und das Format der Fragen. Beantworten Sie jede Frage und jede Antwort im Klartext in einer neuen Zeile. Geben Sie keine Fragenummern an. Hier ist ein Beispiel für zwei Frage-Antwort-Paare:\n\n{test_sample}"
]

for idx, snippet in enumerate(snippets):
    print(snippet)
    output_filename = "data/train.txt" if idx == 0 else "data/test.txt"

    with open(output_filename, "w", encoding='utf-8') as output_file:
        if api_choice == "openai":
                for chunk_idx, chunk in enumerate(chunks):
                    prompt = f"{context}\n\n{chunk}\n\n{snippet}"
                    if process_option == 'one':
                        print(f"\n\n{prompt}")

                    if process_option == 'one' and chunk_idx > 0:
                        break

                    completion = openai.ChatCompletion.create(
                        # model="gpt-4",
                        model="gpt-3.5-turbo-16k",
                        temperature=0,
                        messages=[
                            {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
                            {"role": "user", "content": prompt}
                        ]
                    )

                    response = completion.choices[0].message['content']
                    output_file.write(response + "\n\n")
                    output_file.flush()

                    time.sleep(0.2)
        elif api_choice == "runpod":
            max_tokens = int(model_context_length * 0.9)

        if process_option == 'all':
            with ThreadPoolExecutor(max_workers=8) as executor:
                future_to_chunk = {executor.submit(query_runpod, pod_id, f"{context}\n\n{chunk}\n\n{snippet}", max_tokens): chunk for chunk in chunks}
                for future in concurrent.futures.as_completed(future_to_chunk):
                    chunk = future_to_chunk[future]
                    try:
                        response = future.result()
                    except Exception as exc:
                        print(f"Generated an exception: {exc}")
                    else:
                        # Überprüfen, ob 'response' einen gültigen Wert enthält, bevor Sie versuchen, ihn zu schreiben
                        if response is not None:
                            output_file.write(response + "\n\n")
                            output_file.flush()
                        else:
                            print(f"No response for chunk: {chunk}")
        else:
            for chunk in chunks:
                response = query_runpod(pod_id, f"{context}\n\n{chunk}\n\n{snippet}", max_tokens)
                # Auch hier die Überprüfung
                if response is not None:
                    output_file.write(response + "\n\n")
                    output_file.flush()
                else:
                    print(f"No response for chunk: {chunk}")

                if process_option == 'one':
                    break

!runpodctl stop pod {pod_id}
!runpodctl remove pod {pod_id}

In [ ]:
#@title 6. Q&A zu CSV
import csv
import os
from transformers import AutoTokenizer

# Function to read questions and answers from a text file and write to a CSV file
def q_and_a_to_csv(input_file_path, output_file_path):
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Yarn-Llama-2-7B-128K-GPTQ", use_fast=True)

    max_tokens = 0  # Variable to keep track of the maximum number of tokens in a line

    with open(input_file_path, 'r') as input_file:
        lines = input_file.readlines()

    # Initialize an empty list to hold (prompt, completion) pairs
    prompts_and_completions = []

    # Remove empty lines and strip leading/trailing whitespaces
    lines = [line.strip() for line in lines if line.strip()]

    # Loop through lines two at a time to get question and answer pairs
    for i in range(0, len(lines), 2):
        prompt = lines[i]
        completion = lines[i + 1]

        # Tokenize the lines and update max_tokens if necessary
        prompt_tokens = tokenizer.tokenize(prompt)
        completion_tokens = tokenizer.tokenize(completion)

        max_tokens = max(max_tokens, len(prompt_tokens) + len(completion_tokens))

        prompts_and_completions.append((prompt, completion))

    # Write the list of (prompt, completion) pairs to a CSV file
    with open(output_file_path, 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file, quotechar='"', quoting=csv.QUOTE_ALL)
        csv_writer.writerow(['prompt', 'completion'])  # Write the header row
        csv_writer.writerows(prompts_and_completions)

    print(f"The maximum number of tokens (prompt + completion) in a row of {output_file_path} is {max_tokens}")

# Specify the paths for the input and output files
train_input_file_path = 'data/train.txt'
train_output_file_path = 'data/train.csv'

# Run the function to convert train.txt to train.csv
q_and_a_to_csv(train_input_file_path, train_output_file_path)

# Check if test.txt exists, and if so, convert it to text.csv
text_input_file_path = 'data/test.txt'
text_output_file_path = 'data/test.csv'

if os.path.exists(text_input_file_path):
    q_and_a_to_csv(text_input_file_path, text_output_file_path)
else:
    print("test.txt does not exist, skipping its conversion.")

In [ ]:
#@title 7. CSV zu Huggingface?
from huggingface_hub import HfApi, login
import os

def upload_to_hf_hub(repo_id):
    # Initialize HfApi
    api = HfApi()

    # Define the files to upload
    files_to_upload = ["data/train.csv", "data/test.csv", "data/README.md"]
    uploaded_files = []

    # Upload each file if it exists
    for file_path in files_to_upload:
        if os.path.exists(file_path):
            print(f"Uploading {file_path}...")
            api.upload_file(
                path_or_fileobj=file_path,
                path_in_repo=os.path.basename(file_path),  # Only the filename, not the path
                repo_id=repo_id,
                repo_type="dataset",
            )
            print(f"Uploaded {file_path}.")
            uploaded_files.append(file_path)
        else:
            print(f"{file_path} does not exist, skipping.")

    # Summary
    print("\nSummary:")
    if uploaded_files:
        print("Uploaded files:")
        for file in uploaded_files:
            print(f"- {file}")
    else:
        print("No files were uploaded.")

def main():
    # Login
    print("Logging in to Hugging Face account...")
    login()

    # Get repo path from the user
    repo_id = input("Enter the path to the Hugging Face dataset repo (e.g. username/repo_name): ")

    # Upload files to the repo
    upload_to_hf_hub(repo_id)

if __name__ == "__main__":
    main()